tutorial:
https://ithelp.ithome.com.tw/articles/10243188
https://openhome.cc/Gossip/CodeData/PythonTutorial/DataManagementFunctionsPy3.html
https://openhome.cc/Gossip/CodeData/PythonTutorial/FunctionalProgrammingPy3.

Functional Programming(FP)
target:
1. 避免狀態相依、物件改變
2. function should both be argument and return value
3. be pure and immutable function not impure function to avoid side effect(global var, DOM, log, request...)

eazy sample:

In [3]:
# 資料源
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# 計算function
# 將資料列為參數或內部值/狀態 >> pure function
def sumEvenSquare(dataArray):
    res = 0

    for n in dataArray:
        if n % 2 == 0:
            res += n * n
    
    return res

# 執行 >> 執行多次也不會影響
# 使用 data[:] >> immutable function
res1 = sumEvenSquare(data[:])
print(res1) # 220
res2 = sumEvenSquare(data[:])
print(res2) # 220

220
220


保持彈性，將運算做成proccess function >> higher order function (高階函數)

In [4]:
# 資料源
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

def sum(proccessFn):
    def inner(inputArray):
        res = 0
        for n in inputArray:
            res += proccessFn(n)
        return res
    return inner

# proccess function
def evenSquare(item):
    return item * item if item % 2 == 0 else 0

# 組成function
sumEvenSquare = sum(evenSquare)

# 執行
res = sumEvenSquare(data)
print(res) # 220

220


將proccess function使用lambda取代

In [5]:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

def sum(proccessFn):
    def inner(inputArray):
        res = 0
        for n in inputArray:
            res += proccessFn(n)
        return res
    return inner

res = sum(lambda n: n * n if n % 2 == 0 else 0)(data)
print(res) # 220

220


命令式 (Imperative) v.s 宣告式 (Declarative)
命令式:強調的是執行過程，通常會暴露非常多細節，比較具象

In [6]:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

def sumEvenSquare(dataArray):
    res = 0

    for n in dataArray:
        if n % 2 == 0:
            res += n * n
    
    return res

宣告式：強調的是執行結果，在思考過程中會隱藏細節，比較抽象

In [7]:
import functools as ft

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

def even(data):
    return [*filter(lambda n : n % 2 == 0, data)]

def square(data):
    return [*map(lambda n : n * n, data)]

def sum(data):
    return ft.reduce(lambda s, n : s + n, data, 0)


evenData = even(data)
evenSquareData = square(evenData)
sumResult = sum(evenSquareData)
print(sumResult)

# 若不逐一命名，可以
res = ft.reduce(lambda s, n: s + n, 
        map(lambda n: n * n,
        filter(lambda n : n % 2 == 0, data)), 0)
print(res)

220
220


Curry Function
前情提要：function 本身可以當作另一個 function 的參數傳入，同時也能夠回傳一個新的 function。

延伸：把「設定」和「資料」進行隔離！而這種 function 我們就稱為 Curry Function。

Curry function 有一個非常大的作用，稱為惰性求值(lazy evaluation)，或稱為延遲評估。Lazy evaluation 的重點在於「盡可能延後進行複雜運算的行為」！以上面的例子來說，我們可以透過 add(n) 這種方法需要運算的 function 先行準備好 (此時還沒真的進行運算)，直到需要的時候，才將資料帶入運算。

In [8]:
# a >> 設定
# b >> 資料
def add(a, b):
    return a + b

def add(num, data):
    return num + data

將設定與資料分開，建立一個設定的function，他會回傳一個function，並可以傳入我們要的資料

In [9]:
def add(num):
    def inner(data):
        return num + data
    return inner

接下來就可以建立多個不同的fuinction

In [10]:
pluseOne = add(1)
pluseTwo = add(2)
getNextId = add(1)

把資料帶進去!

In [11]:
data = 1
p1 = pluseOne(data)
p2 = pluseTwo(data)
nextId = pluseOne(data)

print(p1) # 2
print(p2) # 3
print(nextId) # 2

2
3
2


another example 平方 vs 立方

In [12]:
data = [1, 2, 3]

def power(num):
    def inner(data):
        return [*map(lambda x: pow(x, num), data)]
    return inner

square = power(2)
cube = power(3)

print(square(data))
print(cube(data))

[1, 4, 9]
[1, 8, 27]


加入偶數

In [13]:
data = [1, 2, 3]

def even(data):
    return [*filter(lambda n : n % 2 == 0, data)]

def sum(data):
    return ft.reduce(lambda s, n : s + n, data, 0)

def sumEvenPower(powerFn):
    def inner(data):
        evenData = even(data)
        powerData = powerFn(evenData)
        res = sum(powerData)
        return res
    return inner

sumEvenSquare = sumEvenPower(square)
sumEvencube = sumEvenPower(cube)

print(sumEvenSquare(data))
print(sumEvencube(data))

4
8


去除過多的變數命名
1. Compose function
將整個運算流程會使用到的function組合起來作為參數傳入
由於最裡面的function會優先執行，因此傳入的順序為倒續
a(b(c(data))); // 實際上 c 會先被呼叫

In [18]:
data = [1, 2, 3]

def compose(functions):
    def inner(data):
        res = data
        for fn in functions[::-1]:
            res = fn(res)
        return res
    return inner

def sumEvenPower(powerFn):
    # def inner(data):
    #     evenData = even(data)
    #     powerData = powerFn(evenData)
    #     res = sum(powerData)
    #     return res
    # return inner
    return compose([sum, powerFn, even]) # 實際執行與傳入順序相反

sumEvenSquare = sumEvenPower(square)
sumEvencube = sumEvenPower(cube)

print(sumEvenSquare(data))
print(sumEvencube(data))

4
8


閱讀優化 >> 管線化

2. Pipe function

In [17]:
data = [1, 2, 3]

def pipe(functions):
    def inner(data):
        res = data
        for fn in functions:
            res = fn(res)
        return res
    return inner

def sumEvenPower(powerFn):
    # def inner(data):
    #     evenData = even(data)
    #     powerData = powerFn(evenData)
    #     res = sum(powerData)
    #     return res
    # return inner
    return pipe([even, powerFn, sum]) # 實際執行與傳入順序相同

sumEvenSquare = sumEvenPower(square)
sumEvencube = sumEvenPower(cube)

print(sumEvenSquare(data))
print(sumEvencube(data))

4
8


將side effect(ex: log)隔離出來 >>

Tap function

In [21]:
data = [1, 2, 3]

def pipe(functions):
    def inner(data):
        res = data
        for fn in functions:
            res = fn(res)
        return res
    return inner

def tap(function):
    def inner(data):
        function(data)
        return data
    return inner

def printData(data):
    print("current data:", data)

def sumEvenPower(powerFn):
    return pipe([
        even,
        tap(printData),
        powerFn,
        tap(printData),
        sum,
        tap(printData)])

sumEvenSquare = sumEvenPower(square)

print(sumEvenSquare(data))

current data: [2]
current data: [4]
current data: 4
4
